In [4]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt


In [5]:
#Full year 2024

initial_capital = 1000

capital= initial_capital

momentum_path = Path("../data/momentum_data")
signals_path= Path("../data/signals")

signal_files = sorted(signals_path.glob("2024-*.csv"))

for signal_file in signal_files:
    signal_date = pd.to_datetime(signal_file.stem)
    signals = pd.read_csv(signal_file, index_col="Symbol")
    long_tickers = signals[signals["signal"]=="long"].index.to_list()
    short_tickers = signals[signals["signal"]=="short"].index.to_list()
    all_tickers = long_tickers + short_tickers

    if not all_tickers:
        print(f"⚠️ No signals for {signal_date.strftime('%B %Y')}. Carrying forward capital.")
        continue

    allocation_per_stock = capital / len(all_tickers)
    portfolio={}

    #Get signal date from file name
    
    month_end_date = signal_date.replace(day=1) + pd.offsets.MonthEnd(0)

    #Entry(next trading day open)
    for ticker in all_tickers:
        file=momentum_path/f"{ticker}.csv"
        df=pd.read_csv(file, parse_dates=["Date"],index_col="Date")

        valid_date= df.index[df.index > signal_date]
        if valid_date.empty:
            continue

        entry_date = valid_date.min()
        open_price= df.loc[entry_date]["Open"]
        shares = allocation_per_stock/ open_price

        portfolio[ticker]={
            "position": "long" if ticker in long_tickers else "short",
            "price": open_price,
            "shares": shares
        }

    portfolio_value= 0
    long_profit=0
    short_profit=0

    for ticker,data in portfolio.items():
        file = momentum_path/ f"{ticker}.csv"
        df = pd.read_csv(file, parse_dates=["Date"],index_col="Date")

        available_dates = df.index[df.index <= month_end_date]
        if available_dates.empty:
            continue


        exit_date= available_dates.max()
        close_price = df.loc[exit_date]["Close"]
        
        if data["position"] == "long":
            pnl = close_price - data["price"]* data["shares"]
            long_profit+= pnl
            portfolio_value+= allocation_per_stock+pnl

        else: #short
            pnl= (data["price"] - close_price)*data["shares"]
            short_profit+=pnl
            portfolio_value+= allocation_per_stock+pnl

    total_profit= long_profit + short_profit
    capital= portfolio_value

    print(f"📅 {signal_date.strftime('%B %Y')}")
    print(f"  💰 Long P&L:  ${long_profit:.2f}")
    print(f"  📉 Short P&L: ${short_profit:.2f}")
    print(f"  📊 Total P&L: ${total_profit:.2f}")
    print(f"  🧾 Portfolio Value: ${capital:.2f}")
    print("-" * 40)


# Final report
print(f"\n🏁 Final Portfolio Value: ${capital:.2f}")
print(f"📈 Total Return: {((capital / initial_capital - 1) * 100):.2f}%")


📅 January 2024
  💰 Long P&L:  $3118.18
  📉 Short P&L: $31.07
  📊 Total P&L: $3149.24
  🧾 Portfolio Value: $4149.24
----------------------------------------
📅 February 2024
  💰 Long P&L:  $4399.09
  📉 Short P&L: $-55.06
  📊 Total P&L: $4344.03
  🧾 Portfolio Value: $8493.27
----------------------------------------
📅 March 2024
  💰 Long P&L:  $849.77
  📉 Short P&L: $-262.37
  📊 Total P&L: $587.41
  🧾 Portfolio Value: $9080.68
----------------------------------------
📅 April 2024
  💰 Long P&L:  $-707.04
  📉 Short P&L: $189.03
  📊 Total P&L: $-518.01
  🧾 Portfolio Value: $8562.67
----------------------------------------
📅 May 2024
  💰 Long P&L:  $1163.07
  📉 Short P&L: $-88.24
  📊 Total P&L: $1074.83
  🧾 Portfolio Value: $9637.50
----------------------------------------
📅 June 2024
  💰 Long P&L:  $229.55
  📉 Short P&L: $103.77
  📊 Total P&L: $333.32
  🧾 Portfolio Value: $9970.82
----------------------------------------
📅 July 2024
  💰 Long P&L:  $885.82
  📉 Short P&L: $-232.97
  📊 Total P&L